In [ ]:
import os
import re
import csv
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

### Usando Selenium

In [ ]:
urls = [
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/qualificacoes-para-o-mercado-de-trabalho.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/a-apropriacao-cultural-e-o-conhecimento-historico.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/no-mes-de-fevereiro-muitos-paises.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/apropriacao-cultural-e-atualidades.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/representacao-indigena-no-meio-social.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/a-afronta-carnavalesca.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/carnaval-uma-festa-de-cultura.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/carnaval-ou-cultura.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/monopolio-cultural.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/apropriacao-cultural-tao-necessaria-quanto-natural.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/apropriacao-cultural-significa-uma-pessoa.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/dar-voz-para-quem-nao-a-possui.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/carnaval-sinonimo-de-composicao-cultural.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/multiculturalismo-carnavalesco.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/apropriacao-ou-intercambio.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/a-apropriacao-cultural-continua.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/muito-alem-da-fantasia.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/as-fantasias-no-carnaval.htm",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/apropriacao-cultural-e-so-no-carnaval.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/entender-o-outro.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/apropriacao-cultural-e-o-racismo-velado.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/uniao-entre-o-orgao-publico-e-a-opiniao-publica.html",
    "https://educacao.uol.com.br/bancoderedacoes/redacoes/a-democracia-estende-se-ao-stf.html"
]

In [ ]:
# Diretório e nomes dos arquivos
output_dir = "textos_uol"
output_file = os.path.join(output_dir, "dados_redacoes.csv")
processed_links_file = os.path.join(output_dir, "processados.txt")
title_file = os.path.join(output_dir, "titulos.csv")
id_file = os.path.join(output_dir, "ultimo_id.txt")
title_id_file = os.path.join(output_dir, "ultimo_id_titulo.txt")

# Certifica-se de que o diretório existe, e se não, cria-o
os.makedirs(output_dir, exist_ok=True)

# Carrega o último ID salvo para redações ou define como 0
if os.path.exists(id_file):
    with open(id_file, "r") as f:
        last_id = int(f.read().strip())
else:
    last_id = 0

# Carrega o último ID salvo para títulos ou define como 0
if os.path.exists(title_id_file):
    with open(title_id_file, "r") as f:
        last_title_id = int(f.read().strip())
else:
    last_title_id = 0

# Função para gerar o próximo ID incremental para redações
def generate_id():
    global last_id
    last_id += 1
    return f"{last_id:06d}"

# Função para gerar o próximo ID incremental para títulos
def generate_title_id():
    global last_title_id
    last_title_id += 1
    return f"{last_title_id:04d}"

# Carrega os links já processados para evitar duplicação
if os.path.exists(processed_links_file):
    with open(processed_links_file, "r", encoding="utf-8") as f:
        processed_links = set(line.strip() for line in f)
else:
    processed_links = set()

# Carrega os títulos já inseridos para evitar duplicação
existing_titles = {}
if os.path.exists(title_file):
    with open(title_file, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader)  # Ignora o cabeçalho
        for row in reader:
            title_id, title_text = row
            existing_titles[title_text.strip()] = title_id.strip()

# Configura o ChromeDriver no modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Inicializa o navegador Chrome com as opções configuradas
driver = webdriver.Chrome(options=chrome_options)

# Abre o arquivo CSV principal para escrita e define o cabeçalho se necessário
file_mode = "a" if os.path.exists(output_file) else "w"
with open(output_file, mode=file_mode, newline="", encoding="utf-8") as main_file:
    writer_main = csv.writer(main_file)
    
    # Escreve o cabeçalho no CSV principal
    if file_mode == "w":
        writer_main.writerow(["ID", "id_titulo", "Título", "Subtítulo", "Texto da Redação", "Competências", "Notas das Competências", "Comentários das Competências", "Nota Final"])

    # Abre o arquivo de títulos em modo "a" para adicionar títulos sem repetição
    with open(title_file, mode="a", newline="", encoding="utf-8") as title_file_obj:
        writer_title = csv.writer(title_file_obj)
        if not existing_titles:
            writer_title.writerow(["ID", "Título"])  # Cabeçalho para o arquivo de títulos

        # Processa cada URL na lista
        for url in urls:
            if url in processed_links:
                print(f"[INFO] URL já processada, ignorando: {url}")
                continue

            print(f"\n[INFO] Processando URL: {url}")
            time.sleep(5)  # Pausa de 5 segundos antes de cada processamento
            try:
                # Gera um novo ID para a linha atual
                record_id = generate_id()
                print(f"[INFO] ID gerado para o registro: {record_id}")

                # Abre a URL no navegador headless
                driver.get(url)
                print(f"[SUCCESS] Conexão bem-sucedida com {url}")

                # Extrai o título da redação
                try:
                    title_tag = driver.find_element(By.CSS_SELECTOR, "i.custom-title")
                    title = title_tag.text.strip() if title_tag else "Título não encontrado"
                except:
                    title = "Título não encontrado"
                print(f"[INFO] Título extraído: {title}")

                # Verifica se o título já está na lista de títulos inseridos
                if title in existing_titles:
                    title_id = existing_titles[title]  # Usa o ID existente para o título
                else:
                    title_id = generate_title_id()  # Gera um novo ID para o título
                    writer_title.writerow([title_id, title])
                    existing_titles[title] = title_id  # Adiciona o título e ID ao dicionário
                    print(f"[INFO] Título adicionado ao arquivo de títulos: {title}")

                # Extrai o subtítulo da redação
                try:
                    subtitle_tag = driver.find_element(By.CSS_SELECTOR, "section.wording-correction h2")
                    subtitle = subtitle_tag.text.strip() if subtitle_tag else "Subtítulo não encontrado"
                except:
                    subtitle = "Subtítulo não encontrado"
                print(f"[INFO] Subtítulo extraído: {subtitle}")

                # Extrai o texto da redação dentro da tag 'text-composition'
                try:
                    paragraphs = driver.find_elements(By.CSS_SELECTOR, "div.text-composition p")
                    essay_text = " ".join([p.text.strip() for p in paragraphs])
                except:
                    essay_text = "Redação não encontrada"
                print(f"[INFO] Texto da redação extraído.")

                # Extrai as competências e notas
                try:
                    competence_scores_section = driver.find_element(By.CSS_SELECTOR, "section.results-table")
                    competence_scores_list = competence_scores_section.find_elements(By.CSS_SELECTOR, "div.rt-line-option")
                    
                    competences = []
                    competence_scores = []
                    final_score = "Nota final não encontrada"

                    for score in competence_scores_list:
                        topic = score.find_element(By.CSS_SELECTOR, "span.topic").text.strip()
                        points = score.find_element(By.CSS_SELECTOR, "span.points").text.strip()
                        
                        competences.append(topic)
                        competence_scores.append(points)

                    # Verifica e extrai a última nota como "Nota Final" se houver mais de 5 notas
                    if len(competence_scores) > 5:
                        final_score = competence_scores.pop()  # Última nota removida das competências e usada como "Nota Final"
                        competences.pop()  # Remove a última competência correspondente
                except:
                    competences, competence_scores, final_score = [], [], "Erro ao extrair notas"
                print(f"[INFO] Competências e notas extraídas.")

                # Extrai os comentários das competências
                try:
                    competence_comments_section = driver.find_element(By.XPATH, "//h3[text()='Competências']/following-sibling::ul")
                    competence_comments_list = competence_comments_section.find_elements(By.TAG_NAME, "li")
                    competence_comments = [comment.text.strip() for comment in competence_comments_list]
                    concatenated_comments = "; ".join(competence_comments)
                except:
                    concatenated_comments = "Comentários não encontrados"
                
                # Concatena as competências e as notas usando ";" como separador
                concatenated_competences = "; ".join(competences)
                concatenated_scores = "; ".join(competence_scores)

                # Escreve uma linha no CSV principal para cada URL processada
                writer_main.writerow([record_id, title_id, title, subtitle, essay_text, concatenated_competences, concatenated_scores, concatenated_comments, final_score])

                # Marca a URL como processada e salva no arquivo de links processados
                with open(processed_links_file, "a", encoding="utf-8") as f:
                    f.write(url + "\n")
                
                print(f"[SUCCESS] Processado com sucesso: {url}")

            except Exception as e:
                print(f"[ERROR] Erro ao acessar a página {url}: {e}")

# Salva o último ID usado
with open(id_file, "w") as f:
    f.write(str(last_id))

# Salva o último ID de título usado
with open(title_id_file, "w") as f:
    f.write(str(last_title_id))

# Fecha o navegador
driver.quit()

print(f"[INFO] Arquivos CSV salvos em: {output_file} e {title_file}")

### Teste com propostas

In [ ]:
urls = [
    "https://educacao.uol.com.br/bancoderedacoes/propostas/qualificacao-e-o-futuro-do-emprego.html",
    "https://educacao.uol.com.br/bancoderedacoes/propostas/carnaval-e-apropriacao-cultural.html",
    "https://educacao.uol.com.br/bancoderedacoes/propostas/supremo-tribunal-federal-e-opiniao-publica.html"
]

In [ ]:
# Diretório e nome do arquivo CSV
output_dir = "textos_uol"
title_file = os.path.join(output_dir, "titulos.csv")

# Lista para armazenar os textos das propostas extraídos
textos_propostas = []

print("Iniciando extração de textos das propostas...")

# Loop para processar cada URL e extrair o texto da proposta
for index, url in enumerate(urls):
    print(f"\nProcessando URL {index + 1}/{len(urls)}: {url}")
    try:
        # Faz a requisição para obter o conteúdo da página
        response = requests.get(url)
        response.raise_for_status()
        
        # Cria o objeto BeautifulSoup para analisar o HTML da página
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Localiza a cadeia de divs: article-wording -> calc-height -> image-content-pad -> text
        article_wording = soup.find('div', class_='article-wording')
        if article_wording:
            calc_height = article_wording.find('div', class_='calc-height')
            if calc_height:
                image_content_pad = calc_height.find('div', class_='image-content-pad')
                if image_content_pad:
                    text_div = image_content_pad.find('div', class_='text')
                    
                    # Extrai o conteúdo das tags <p> dentro de "text"
                    if text_div:
                        paragraphs = text_div.find_all('p', recursive=False)
                        text_proposal = " ".join(p.get_text(strip=True) for p in paragraphs)
                        print(f"Texto extraído com sucesso da URL {index + 1}")
                    else:
                        text_proposal = "Seção de texto não encontrada na div 'text'."
                        print(f"Alerta: {text_proposal}")
                else:
                    text_proposal = "Div 'image-content-pad' não encontrada."
                    print(f"Alerta: {text_proposal}")
            else:
                text_proposal = "Div 'calc-height' não encontrada."
                print(f"Alerta: {text_proposal}")
        else:
            text_proposal = "Div 'article-wording' não encontrada."
            print(f"Alerta: {text_proposal}")

        # Adiciona o texto extraído à lista de resultados
        textos_propostas.append(text_proposal)

    except requests.exceptions.RequestException as e:
        error_message = f"Erro ao acessar a URL {url}: {e}"
        print(error_message)
        textos_propostas.append("Erro ao acessar a página")

print("\nExtração de propostas concluída.")

# Verifica se o arquivo CSV existe e lê os dados
if os.path.exists(title_file):
    print(f"Lendo arquivo CSV existente: {title_file}")
    with open(title_file, mode="r", newline="", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames
        rows = list(reader)
    print("Leitura concluída.")

    # Adiciona a coluna "Textos Base" se não estiver presente
    if "Textos Base" not in fieldnames:
        print("Adicionando coluna 'Textos Base' ao CSV.")
        fieldnames.append("Textos Base")
        for row in rows:
            row["Textos Base"] = ""  # Inicializa a coluna para cada linha existente
else:
    print(f"Erro: O arquivo {title_file} não existe.")
    exit()

# Adiciona os textos extraídos na coluna "Textos Base" na ordem dos URLs
print("Atualizando o CSV com os textos extraídos...")
for i, row in enumerate(rows):
    if i < len(textos_propostas):
        row["Textos Base"] = textos_propostas[i]

# Escreve as mudanças de volta no arquivo CSV
with open(title_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print(f"\nArquivo '{title_file}' atualizado com os textos das propostas na coluna 'Textos Base'.")